In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from geotools.path_calc import address_search, get_metro_bounds
from geotools.weighting import weighting_function


In [ ]:
metro_grid = pd.read_csv("metro_grid.csv", sep=";")

In [ ]:
metro_grid

In [ ]:
lat, lng = [45.147762, -93.134160]
lng, lat = metro_grid.loc[10000,:]
address_data = address_search(lng, lat, 5000.)

In [ ]:
address_data

In [ ]:
df = address_data
class_weights = {
    'AGRICULTURAL': 0.5,
    'APARTMENTS / COOP': 10,
    'COMMERCIAL / GOLF COURSE': 1,
    'CONDOMINIUMS': 5,
    'DOUBLE BUNGALOWS': 3,
    'INDUSTRIAL': 3,
    'MANUFACTURED HOME PARK': 10,
    None: 0,
    'RESIDENTIAL DUPLEXES': 3,
    'RESIDENTIAL SINGLE FAMILY': 2,
    'SEASONAL RECREATIONAL': 1,
    'TAX FORFEIT': 0,
    'UTILITY PROPERTY': 0
}
class_weights = {
    '100 Res 1 unit': 2,
    '105 Res 2-3 units': 2.5,
    '110 Apt 4+ units': 6,
#     '125 SRR',
#     '140 Res V Land',
#     '200 Agricultural',
#     '211 Rural Vacant Land',
    '300 Commercial': 2,
    '305 Industrial': 1.5,
#     '320 Q Golf Course',
    '901 Schools-Public': 1,
#     '911 Cemetery-Public',
#     '912 Cemetery-Private',
#     '915 Church',
#     '916 Church-Residence',
#     '917 Church-Other Res',
#     '918 Church - Other',
#     '925 Trans Housing',
#     '931 Charit Inst',
#     '940 Wetlands',
#     '941 ForestParkRefuge',
#     '942 Indian Resrv',
#     '951 Federal Property',
#     '952 State Property',
#     '955 Co Srvc Other',
#     '956 Muni Srvc Ent',
#     '958 Muni Srvc Other',
#     '962 T E Misc Co D 3',
#     '981 State Acquired',
#     '982 State Admin-DNR',
#     '983 Co Admin-TaxForf',
#     '990 InLieuTx Co D 1',
}

year_weights = lambda x: np.exp((x - 2020) / 10)
use_weight = df["USECLASS1"].map(lambda x: class_weights.get(x, 0)).values
d = use_weight * year_weights(df["YEAR_BUILT"].values)

In [ ]:
from functools import reduce
# reduceaddress_data["USECLASS1"].unique()
use_classes = list(reduce(lambda a, b: a + b, [address_data[f"USECLASS{i}"].unique().tolist() for i in range(1,5)]))
set(use_classes)

In [ ]:
class_weights = {
    'AGRICULTURAL': 0.5,
    'APARTMENTS / COOP': 10,
    'COMMERCIAL / GOLF COURSE': 1,
    'CONDOMINIUMS': 5,
    'DOUBLE BUNGALOWS': 3,
    'INDUSTRIAL': 3,
    'MANUFACTURED HOME PARK': 10,
     None: 0,
    'RESIDENTIAL DUPLEXES': 3,
    'RESIDENTIAL SINGLE FAMILY': 2,
    'SEASONAL RECREATIONAL': 1,
    'TAX FORFEIT': 0,
    'UTILITY PROPERTY': 0
}

In [ ]:
x = np.linspace(1950, 2020, 1000)
year_weights = lambda x: np.exp((x - 2020) / 10)
fig, ax = plt.subplots()
ax.plot(x, year_weights(x), "--")
plt.show()

In [ ]:
def weighting_function(df):
    use_weight = df["USECLASS1"].map(class_weights)
    return use_weight * year_weights(df["YEAR_BUILT"])

In [ ]:
address_data["WEIGHT"] = weighting_function(address_data)

In [ ]:
sns.histplot(address_data["WEIGHT"])

In [ ]:
sns.scatterplot(y="WEIGHT", x="DISTANCE", data=address_data)

In [ ]:
address_data["WEIGHT"] = address_data["USECLASS1"].apply(lambda x: class_weights[x]) *


In [ ]:
np.unique(address_data["YEAR_BUILT"])

In [ ]:
new_mask = address_data["YEAR_BUILT"] > 0
address_data.loc[new_mask].plot.scatter(y="WEIGHT", x="YEAR_BUILT")